In [ ]:
import os
import pandas
import tarfile
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers[0].setFormatter(logging.Formatter('%(asctime)s: %(message)s'))

locations = {
    'gfs': '/l/cnets/datasets/Telecom_BDC_2015',
    'diskstation': '/media/diskstation/Datasets/Telecom Big Data Challenge 2015',
    'data': os.path.expanduser('~/data/tbdc15'),
    'hdd': '/media/giovanni/Multimedia/Datasets/Telecom Big Data Challenge 2015',
    'repo': os.path.expanduser('~/repos/tbdc15')
}


def getpaths(city, loc='gfs', boxesloc=None, storeloc=None):
    root = locations[loc]
    city_codes = {'RO': 'RM'}
    code = city[:2].upper()
    if code in city_codes:
        code = city_codes[code]
    paths = {
        'trips': '{root}/infoblu/{city}.tar.gz'.format(root=root, city=city),
        'accidents': '{root}/unipol/BDC2015_UnipolsaiClaims2014_{city}.csv'.format(root=root, city=code),
        'boxes': '{root}/city_boxes.csv'.format(root=root if boxesloc is None else locations[boxesloc]),
        'store': '{root}/trip_accidents_store.hdf'.format(root=root if storeloc is None else locations[storeloc])
    }
    return paths


def getbox(path, city):
    city_code = city[0].lower()
    df_box = pandas.read_csv(path, index_col='city')
    df_box.ix[city_code]
    box = df_box.ix[city_code].to_dict()
    return box


def read_trips(path, box, scale=1000.0, break_at=None):
    index_columns = ['i', 'j', 'weekday', 'hour']
    trips = pandas.DataFrame(columns=index_columns + ['trips', 'trips_start']).set_index(index_columns)

    # set break_at to an integer and it will stop exactly after that number of iterations
    i = 0

    with tarfile.open(path, mode='r:gz') as tf:
        # open tar file in random access mode with on-the-fly gzip decompression
        for member in tf:
            if break_at is not None and i == break_at:
                break
            i += 1

            # read contents of TAR archive. Each file in the archive contains 
            # the data of a different day.
            logger.info(member.name)
            f = tf.extractfile(member)

            # do not use the "type" and "speed" columns, since we don't need them. This saves memory.
            df = pandas.read_csv(f, 
                                 names=['trip', 'timestamp', 'lat', 'lon', 'type', 'speed'],
                                 usecols=['trip', 'timestamp', 'lat', 'lon'],
                                 sep=';', 
                                 parse_dates=['timestamp'])

            # compute the cell, weekday, and hour
            df['i'] = ((df['lat'] - box['lat_min']) * scale).round()
            df['j'] = ((df['lon'] - box['lon_min']) * scale).round()
            df['weekday'] = df['timestamp'].map(pandas.Timestamp.weekday)
            df['hour'] = df['timestamp'].map(lambda k: k.hour)

            # count how many trips in each cell, weekday, hour and append. 
            # Note that the first group-by returns a series object, 
            # and we wrap this into a DataFrame.        
            s1 = df.filter(index_columns).groupby(index_columns).apply(len)

            # do the same but only considering the first frame of each trip.
            df_ff = df.groupby('trip', as_index=False).head(1)
            s2 = df_ff.filter(index_columns).groupby(index_columns).apply(len)

            df = pandas.DataFrame({'trips': s1, 'trips_start': s2})
            
            trips = trips.append(df)

    return trips


def read_accidents(path, box, scale=1000.0):
    index_columns = ['i', 'j', 'weekday', 'hour']    
    df = pandas.read_csv(path)
    df.rename(columns={'day_type': 'weekday', 'time_range': 'hour'}, inplace=True)
    df['i'] = ((df['latitude'] - box['lat_min']) * scale).round()
    df['j'] = ((df['longitude'] - box['lon_min']) * scale).round()
    s = df.groupby(index_columns).apply(len)
    accidents = pandas.DataFrame({'accidents': s})
    return accidents


def make_city_frame(city, 
                    loc='frosty', 
                    boxesloc='frosty',
                    storeloc='frosty',
                    scale=1000.0, 
                    break_at=None):
    """
    Reads data of trips and accidents and store data frame into HDF format
    """
    paths = getpaths(city, loc=location, boxesloc=boxesloc, storeloc=storeloc)
    box = getbox(paths['boxes'], city)
    logger.info("Reading trips...")
    trips = read_trips(paths['trips'], box, scale=scale, break_at=break_at)
    logger.info("Reading accidents...")
    accidents = read_accidents(paths['accidents'], box, scale=scale)
    logger.info("Storing data...")
    joined_df = trips.join(accidents).fillna(0).reset_index()
    joined_df.to_hdf(paths['store'], city, complib='blosc', complevel=6)
    logger.info("Data saved to HDF:".format(paths['store']))

# Create dataset

## Select city 

In [ ]:
cities = ['bari', 'milano', 'napoli', 'palermo', 'roma', 'torino', 'venezia']
location = 'gfs'
boxes_location = 'data'
store_location = 'data'
scale = 1000.0

## Read data

Use the following to remove an existing store file, if needed. (Use `C-M y` to make the cell runnable).

rm -f /u/gciampag/data/tbdc15/trip_accidents_store.hdf

In [ ]:
ll -h ~/data/tbdc15/

In [ ]:
for city in cities:
    logging.info("City: {}".format(city))
    make_city_frame(city, 
                    loc=location, 
                    scale=scale, 
                    boxesloc=boxes_location,
                    storeloc=store_location, 
                    break_at=None)

<hr style="height: .1em"/>

# Plot the data

In [ ]:
%matplotlib inline
import numpy
from pylab import *

# scatter plot
def scatter_trips_accidents(df, city, use_trips_starts=False):
    if use_trips_starts:
        xcol = 'trips_start'
    else:
        xcol = 'trips'
            
    df.plot(x=xcol, y='accidents', kind='scatter', marker='x', alpha=.2, color='k')

    # trend line
    xmin = max(df[xcol].min(), 1)
    xmax = df[xcol].max()
    bins = numpy.logspace(numpy.log10(xmin), numpy.log10(xmax), 20)
    df.groupby(numpy.digitize(df[xcol], bins=bins)).mean().plot(x=xcol, y='accidents',
                                                                color='r', linestyle='solid',
                                                                marker='o', ax=gca(), alpha=.5, 
                                                                linewidth=2)
    grid('off')
    title(city)
    if use_trips_starts:
        xlabel('Traffic (start of trip)')
    else:
        xlabel('Traffic')
    ylabel('Accidents')
    xscale('log')
    tight_layout()
    savefig('trips_accidents_scatter_{}.pdf'.format(city))
    savefig('trips_accidents_scatter_{}.png'.format(city))


def hist_accidents(df, city):
    hist(df['accidents'], log=True, bins=60, color='white')
    ylim(.1, ylim()[1])
    xlabel('Accidents')
    ylabel('Frequency')
    title(city)
    tight_layout()
    savefig('accidents_histogram_{}.pdf'.format(city))
    savefig('accidents_histogram_{}.png'.format(city))

In [ ]:
for city in cities:
    paths = getpaths(city, 
                     loc=location, 
                     boxesloc=boxes_location,
                     storeloc=store_location)
    df = pandas.read_hdf(paths['store'], city)
    df = df.groupby(['i', 'j']).sum().filter(['trips', 'trips_start', 'accidents'])
    scatter_trips_accidents(df, city)
    scatter_trips_accidents(df, city, use_trips_starts=True)
    hist_accidents(df, city)
    print city

## Scatter plot of trips vs trip starts 